In [1]:
import pandas as pd
import ast

# Load your CSV
df = pd.read_csv('/home/priya/Desktop/fyp/Src alwin/Src/top5_transformer_configurations.csv')

# Extract all needed config values directly from the configuration column
def extract_config(row):
    try:
        config = ast.literal_eval(row['configuration'])
        trans = config.get('transformer', {})
        return pd.Series({
            'activation': trans.get('activation'),
            'hidden_dim': trans.get('dim_feedforward'),
            'hidden_layers': trans.get('hidden_layers'),
            'batch_size': trans.get('batch_size'),
            'loss_fn': trans.get('loss'),
            'val_r2': row.get('val_r2'),
            'val_rmse': row.get('val_rmse'),
            'test_r2': row.get('test_r2'),
            'test_rmse': row.get('test_rmse')
        })
    except Exception as e:
        print("Error parsing row:", e)
        return pd.Series()

# Apply extraction
final_df = df.apply(extract_config, axis=1)
final_df.index = [f'Trial {i+1}' for i in range(len(final_df))]

# Fill any missing values (optional)
final_df['hidden_layers'] = final_df['hidden_layers'].fillna(3)
final_df['batch_size'] = final_df['batch_size'].fillna(64)
final_df['loss_fn'] = final_df['loss_fn'].fillna('mse')

# Clean formatting
final_df['val_rmse'] = final_df['val_rmse'].round(4)
final_df['test_rmse'] = final_df['test_rmse'].round(4)
final_df['val_r2'] = final_df['val_r2'].round(4)
final_df['test_r2'] = final_df['test_r2'].round(4)

# Display styled table
styled = final_df.style.set_caption("Full Transformer Configurations and Performance") \
    .background_gradient(subset=['val_rmse', 'test_rmse'], cmap='coolwarm') \
    .format({'val_rmse': "{:.4f}", 'test_rmse': "{:.4f}", 'val_r2': "{:.4f}", 'test_r2': "{:.4f}"}) \
    .set_properties(**{'text-align': 'center'})

styled


,activation,hidden_dim,hidden_layers,batch_size,loss_fn,val_r2,val_rmse,test_r2,test_rmse
Trial 1,gelu,256,3,64,mse,-4.6000,0.0436,0.9098,0.1068
Trial 2,relu,256,3,64,mse,-5.4187,0.0467,0.9005,0.1121
Trial 3,relu,256,3,64,mse,-5.4187,0.0467,0.9005,0.1121
Trial 4,gelu,128,3,64,mse,-4.4762,0.0431,0.8983,0.1134
Trial 5,relu,256,3,64,mse,-6.0008,0.0488,0.8934,0.1160
